In [19]:
import os
import sys
import math
import bisect
import argparse
import gzip
import numpy as np
from collections import namedtuple
from methylbed_utils import MethRead,make_coord,bed_to_coord,coord_to_bed
import pysam
from Bio import SeqIO
import re
import multiprocessing as mp
import time
start_time = time.time()


bampath = "/kyber/Data/Nanopore/projects/regot_targ/082720mamhelen/bam/082720mamhelen.bam"
outbam = "test_out.bam"
cpgpath = "/kyber/Data/Nanopore/Analysis/gmoney/CHM13/final_assembly/whole_genome/CpGmethylation.bed.gz"
#fastafile = "/kyber/Data/Nanopore/Analysis/gmoney/CHM13/final_assembly/ref/t2t-chm13.20200727.fasta"
fastafile = "/atium/Data/old_mithril_ref/mouse/GCF_000001635.26_GRCm38.p6_genomic.fna"


In [20]:

def get_windows_from_bam(bampath,winsize = 100000) :
    with pysam.AlignmentFile(bampath,"rb") as bam :
        stats = bam.get_index_statistics()
        lengths = bam.lengths
        coords = list()
        for stat,contigsize in zip(stats,lengths) :
            # only fetch from contigs that have mapped reads
            if stat.mapped == 0 :
                continue
            numbins = math.ceil(contigsize/winsize)
            wins = [ [x*winsize+1,(x+1)*winsize] for x in range(numbins) ]
            wins[-1][1] = contigsize
            for win in wins :
                coords.append(make_coord(stat.contig,win[0],win[1]))
    return coords

window = get_windows_from_bam(bampath)[1]
window

'NC_000067.6:100001-200000'

In [38]:

def reset_bam(bam,genome_seq) :
    try : 
        refseq = bam.get_reference_sequence()
    except ValueError :
        try : 
            # MD tag not present in minimap2
            refseq = genome_seq[ 
                    bam.reference_start:
                    bam.reference_end]
        except :
            print("supply the reference genome (-f,--fasta)",file=sys.stderr)
            sys.exit()
    bam.query_sequence = refseq.upper()
    bam.cigarstring = ''.join([str(len(refseq)),"M"])
    return bam


In [39]:
bam = pysam.AlignmentFile(bampath,"rb")
fasta = pysam.FastaFile(fastafile)

newbam = reset_bam(bam,fasta)


AttributeError: 'pysam.libcalignmentfile.AlignmentFile' object has no attribute 'get_reference_sequence'